In [1]:
from omniwatermask import extract_water
from pathlib import Path
from s2mosaic import mosaic

In [2]:
import logging

logging.basicConfig(level=logging.INFO)

Use s2mosaic to create a cloud and cloud shadow free Sentinel-2 mosaic


In [3]:
mosaic_path = mosaic(
    grid_id="16TCP",
    start_year=2022,
    start_month=6,
    duration_months=2,
    output_dir=Path("example"),
    required_bands=["B04", "B03", "B02", "B08"],  # OWM requires RGB+NIR bands
    overwrite=False,
)

Run the water inference


In [4]:
water_mask_path = extract_water(
    scene_paths=[mosaic_path],  # you can pass a list of images
    band_order=[1, 2, 3, 4],  # band order of the input images, expects RGB+NIR
    overwrite=True,
    batch_size=1,
    inference_dtype="bf16",
)

Using mps:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:root:Exporting to example/16TCP_2022-06-01_to_2022-08-01_valid_data_mean_B04_B03_B02_B08_OmniWaterMask_0.1.2.tif
INFO:root:Processing 16TCP_2022-06-01_to_2022-08-01_valid_data_mean_B04_B03_B02_B08.tif
INFO:root:Predicting water mask for 16TCP_2022-06-01_to_2022-08-01_valid_data_mean_B04_B03_B02_B08.tif
INFO:root:Integrating water detection methods
INFO:root:Building vector prior in thread
INFO:root:Building negative priors in thread
INFO:root:Found matching GeoDataFrame in cache
INFO:root:Predicting water mask using custom model
INFO:root:Loading GeoDataFrame from /Users/nick/Documents/Work code/OmniWaterMask/water_vectors_cache/gdfs/e40e71d2-0486-4162-a2d5-46c4ea5144f9.parquet
INFO:root:No matching GeoDataFrame found in cache


True False False
False True True


INFO:root:Combining vector priors
INFO:root:Number of non point features: 316354
INFO:root:Buffering line features
INFO:root:Added GeoDataFrame to cache
INFO:root:Waiting for vector priors to finish
INFO:root:Waiting for negative priors to finish
INFO:root:Optimising NDWI
INFO:root:Multi-scale optimisation accuracy finished
INFO:root:Optimising model predictions
INFO:root:Exporting 16TCP_2022-06-01_to_2022-08-01_valid_data_mean_B04_B03_B02_B08.tif to example/16TCP_2022-06-01_to_2022-08-01_valid_data_mean_B04_B03_B02_B08_OmniWaterMask_0.1.2.tif
INFO:rasterio._err:GDAL signalled an error: err_no=1, msg='PROJ: internal_proj_create_from_name: /opt/homebrew/anaconda3/envs/py312/share/proj/proj.db contains DATABASE.LAYOUT.VERSION.MINOR = 2 whereas a number >= 3 is expected. It comes from another PROJ installation.'


In [6]:
import rasterio as rio
from rasterio.windows import Window
from matplotlib import pyplot as plt
import numpy as np

Load the full scene at 100m resolution


In [ ]:
visual_array = rio.open(mosaic_path).read([1, 2, 3], out_shape=(3, 1098, 1098))
water_mask = rio.open(water_mask_path[0]).read(1, out_shape=(1, 1098, 1098))

fig, ax = plt.subplots(1, 2, figsize=(30, 15))
ax[0].imshow(np.clip((visual_array.transpose(1, 2, 0) / 4000), 0, 1))
ax[1].imshow(np.clip((visual_array.transpose(1, 2, 0) / 4000), 0, 1))
ax[1].imshow(water_mask, cmap="coolwarm_r", alpha=0.6, interpolation="nearest")

Read in a small window to view in more detail


In [ ]:
window_size = 1500
window = Window(
    col_off=10980 - window_size, row_off=0, width=window_size, height=window_size
)

with rio.open(mosaic_path) as mosaic_ds:
    visual_array = mosaic_ds.read([1, 2, 3], window=window)

with rio.open(water_mask_path[0]) as mask_ds:
    water_mask = mask_ds.read(1, window=window)

fig, ax = plt.subplots(1, 2, figsize=(30, 15))

ax[0].imshow(np.clip((visual_array.transpose(1, 2, 0) / 4000), 0, 1))
ax[1].imshow(np.clip((visual_array.transpose(1, 2, 0) / 4000), 0, 1))
ax[1].imshow(water_mask, cmap="coolwarm_r", alpha=0.6, interpolation="nearest")